# Web Scraping "Página 12"
## Primer paso: request
### **Importar Librerías**
Lo primero que vamos a realizar es importar las librerías, en este caso vamos a necesitar requests para conectar la url y bs para realizar el scrapping. Aparte importamos pandas para crear y manipular un dataset con la información obtenida. Importamos además matplotlib para realizar lagunas gráficas.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### **Establecer conexión con la url**
Lo siguiente que vamos a realizar una solicitud a la url del portal del diario página12, para ello utilizamos de la libreria requests que importamos en las primeras líneas el método 
```python 
.get()
```

In [2]:
url = 'https://www.pagina12.com.ar/'
p12 = requests.get(url)

Para chequear que la solicitud fue establecida con éxito, ejecutamos:
    
```python
.status_code

```
Si está todo ok nos devería volver 200, para más info buscar sobre códigos de status HTTP

In [3]:
p12.status_code


200

Ahora que ya establecimos la conexión con la URL, vamos a imprimir el contenido html de la página con la comando:
```python
.text
```

In [5]:
print(p12.text)

<!DOCTYPE html><html class="no-js " lang="es"><head><meta charset="utf-8"><title>Página | 12: La otra mirada sobre Argentina y el mundo</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><link rel="canonical" href="https://www.pagina12.com.ar"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Información sobre Argentina y el mundo. Noticias en fotos y videos de los principales hechos y acontecimientos del país. Análisis, opinión y entrevistas."><meta property="fb:pages" n

## Segundo paso: BeautifulSoup
### Limpiar el texto html de la página
Lo siguiente que tenemos que hacer es extraer la información del código html que nos parezca relevante. Para ello vamos a utilizar la librería BeautifulSoup.\
Con esta librería lo que vamos a realizar es trozar nuestro código html y quedarnos con las partes que contengan la información que querramos.
Lo primero que hay que hacer es crear un objeto soup, lo hacemos con el método:
```python
BeautifulSoup(web_scrapear.text, 'lmxl')
 ```

In [6]:
# Lo guardamos en una variable
sopa = BeautifulSoup(p12.text, 'lxml')

In [7]:
# Chequeamos el tipo del dato
type(sopa)

bs4.BeautifulSoup

In [8]:
# Imprimos el contenido que guardamos en la variable y aplicamos método .prettify() para verlo más ordenado
print(sopa.prettify())

<!DOCTYPE html>
<html class="no-js" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Página | 12: La otra mirada sobre Argentina y el mundo
  </title>
  <meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/>
  <link href="https://www.pagina12.com.ar" rel="canonical"/>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <meta content="Información sobre Argentina y el mundo. Noticias en fotos y videos de los principales hechos y acontecimientos del país. Análisis, opinión y entrevistas." name="description" property="des

### Seleccionamos y extraemos la información que queremos screapear
Lo siguiente que vamos a realizar es seleccionar la información que queremos escrapear, en este caso deseamos selecionar todas las secciones que nos ofrece el portal para realizar un scraping de todos los articulos que contienen cada una de ellas.\
Para extraer la información lo que vamos a hacer es utilizar el método .find(), con el cual vamos a poder definir que tipo de etiqueta o tag queremos extraer. Luego sobre el objeto que crea el método .find(), lo que podemos hacer es usar el método .find_all ('tag').

```python
.find('div', attrs={'class' : 'aca la caracteristica de ese class'}).find_all('tag')
```

In [9]:
# Extraemos las secciones y las guardamos en una variable
secciones = sopa.find('ul', attrs={'class' : 'horizontal-list main-sections hide-on-dropdown'}).find_all('li')

In [10]:
# Guardamos el primer elemento para poder trabajarlo más facilmente y luego replicarlo a las demás secciones
seccion = secciones[0]
# Visualizamos el contenido
print (seccion)

<li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>


Analizamos lo que nos devuelve y observamos que nos interesan dos cosas, por un lado el título de la sección, en este caso "El país", y por otro lado la URL, la cual nos va a permitir a nosotros poder automatizar el programa de scraping.\
Para extraer esa información vamos a utilizar el método .get_text() para extraer la información del título de la sección y el método .get('href') para extraer la dirección de la url
```python
.get_text()
.get('href')
```

In [11]:
# Extraemos el nombre de la sección.
seccion.a.get_text()

'El país'

In [12]:
# Extraemos la URL
seccion.a.get('href')

'https://www.pagina12.com.ar/secciones/el-pais'

### **Creamos una lista con todas las secciones**
Ahora que ya extrajimos la información de una sección, podemos replicar lo mismo para las demás secciones.\
Para ello vamos a crear una lista que contenga cada una de ellas, accedemos a cada una de ellas y nos quedamos con el atributo ('href) y lo replicamos con un ciclo for a las demás listas que guardamos en la variables secciones.

In [13]:
# Creamos la lista, 
links_secciones = [seccion.a.get('href') for seccion in secciones]

In [14]:
# Imprimimos y chequeamos que ya haya salido todo ok
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo']

### **Extraemos las notas de cada seccion**
Ahora que ya tenemos todas las url que necesitamos de cada una de las secciones, procedemos a extraer las notas que contienen cada una de ellas. Para ello volvemos a quedarnos con un elemento para luego replicarlo a los demás.


In [15]:
# Guardamos al primer elemento en una variable, con un nuevo request, luego pasamos el mismo método a toda la lista de links
sec = requests.get(links_secciones[0])

In [16]:
# Chequeamos que este todo ok con la conexión, debe devolver 200
sec.status_code

200

### Extraer texto de las notas
Ahora lo que vamos a realizar es extraer las url a las notas.\
Debemos repetir el proceso que hicimos anteriormente, por lo cual creamos una sopa de la sección, y luego analizamos el contenido que queremos extraer. \


In [17]:
# Lo que tenemos que hacer ahora es volver a partir el elemento con el BeautifulSoup, de tla manera creamos un objeto soup de la sección
sopa_seccion = BeautifulSoup(sec.text, 'lxml')

In [18]:
# Imprimimos para ver que tenemos
print (sopa_seccion.prettify())

<!DOCTYPE html>
<html amp="" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   El país | Página12
  </title>
  <!-- DUST PATH: /usr/src/app/src/widgets/fc_jsonLD.dust/ -->
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "NewsArticle","mainEntityOfPage": {"@type": "WebPage","@id": "https://www.pagina12.com.ar/secciones/el-pais"},"headline": "El país | Página12","isAccessibleForFree": "False","image": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/lo

* Si observamos la web de página12 podemos obserrvar que tenemos una noticia principal que es la que aparece primero y en mayor dimensión. Vemos que se encuentra dentro de ```<div>```y corresponde a una clase  ```<article-item_content>```.\
* Vamos a extraer esa información del artículo promocionado y la vamos a guardar en una variable.

In [19]:
# Extraemos la información del artículo promocionado y lo guardamos en una variable con el método find y find_all,
art_top = sopa_seccion.find('div', attrs={'class' : "article-item__content-footer-wrapper gutter-small deco-bar-here-left is-mobile-top"})

In [20]:
print (art_top)

<div class="article-item__content-footer-wrapper gutter-small deco-bar-here-left is-mobile-top"><div class="article-item__content"><!-- Title --><h2 class="h1 title-list"><a href="/499903-la-apuesta-del-kirchnerismo-un-salto-del-pasado-al-futuro">La apuesta del kirchnerismo: un salto del pasado al futuro</a></h2><!-- Kicker --><p><a href="/499903-la-apuesta-del-kirchnerismo-un-salto-del-pasado-al-futuro">El armado de cara a 2023, después del acto de La Plata</a></p></div><div class="article-item__footer"><!-- Author --><div class="author hide-on-mobile"><span>Por </span><a href="https://www.pagina12.com.ar/autores/870-karina-micheletto">Karina Micheletto</a></div><!-- Date --><div class="date hide-on-mobile">21 de noviembre de 2022</div><!-- Author Mobile --><div class="author is-display-inline p12-separator-right-gray hide-on-desktop"><span>Por </span><a href="https://www.pagina12.com.ar/autores/870-karina-micheletto">Karina Micheletto</a></div><!-- Date Mobile --><div class="date is-

* Advertimos que también existen dos artículos más que podríamos llamarlos márginales, que también están patrocinados, por lo cual, vamos a guaradrlos en una variable.

In [21]:
sopa_seccion.find('div', attrs={'class' : "articles-list is-grid-col2 grid-mobile-row"})

<div class="articles-list is-grid-col2 grid-mobile-row"><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/lists/articles_list_item.dust/ --><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/articles/featured_article.dust/ --><article class="article-item article-item--featured"><!-- Image --><div class="article-item__header deco-bar-here-bottom is-mobile-left"><a href="/499895-argentina-recupero-archivos-diplomaticos-relacionados-con-la"><!-- DUST PATH: /usr/src/app/node_modules/frontend-core/views/widgets/fc_displayImg_amp.dust/ --><amp-img alt="La secretaria general del Ministerio de Relaciones Exteriores francés, Anne-Marie Descótes, le entregó el material al vicecanciller Pablo Tettamanti." class="" height="313" layout="responsive" src="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/670312-fotos-20documentos-20dh.jpg?itok=jVA0BlUM" srcset="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/670312-fotos-20documentos-20dh.jpg?ito

In [22]:
art_marginales = sopa_seccion.find('div', attrs={'class' : "articles-list is-grid-col2 grid-mobile-row"})

In [23]:
art_marginales

<div class="articles-list is-grid-col2 grid-mobile-row"><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/lists/articles_list_item.dust/ --><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/articles/featured_article.dust/ --><article class="article-item article-item--featured"><!-- Image --><div class="article-item__header deco-bar-here-bottom is-mobile-left"><a href="/499895-argentina-recupero-archivos-diplomaticos-relacionados-con-la"><!-- DUST PATH: /usr/src/app/node_modules/frontend-core/views/widgets/fc_displayImg_amp.dust/ --><amp-img alt="La secretaria general del Ministerio de Relaciones Exteriores francés, Anne-Marie Descótes, le entregó el material al vicecanciller Pablo Tettamanti." class="" height="313" layout="responsive" src="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/670312-fotos-20documentos-20dh.jpg?itok=jVA0BlUM" srcset="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2022-11/670312-fotos-20documentos-20dh.jpg?ito

* Ahora lo que vamos a hacer es extraer el contenido de todas las demás listas de noticias que aparecen más abajo, y nuevamente vamos a guardarlo en una variable.

In [24]:
art_lista = sopa_seccion.find('article', attrs={'class' : "article-item article-item--teaser"})

In [25]:
print(art_lista.a.get('href'))

/499879-los-aliados-del-pro-y-un-intento-de-borrar-la-memoria


### Crear función para extraer todo el contenido que nos interese de cada nota
Ahora vamos a crear una función que nos devuelva el contenido de nos interesa de cada noticia, en este caso la url para luego automatizar el scrap.

In [26]:
def extraer_notas (sopa):
    '''
    Función que relice un objeto de BeautifulSoup de una página de secciones
    y devuelve una lista de URLs a las notas de esa sección
    '''
    lista_notas = []
    
    # Obtengo el artículo promocionado
    art_promocionado = sopa.find ('div', attrs={'class' : "article-item__content-footer-wrapper gutter-small deco-bar-here-left is-mobile-top"})
    for promocionado in art_promocionado.find_all ('a'):
        lista_notas.append(promocionado.get('href'))
    
    # Obtengo los artículos marginales
    articulos_marginales = sopa.find ('div', attrs={'class' : "articles-list is-grid-col2 grid-mobile-row"})
    for link in articulos_marginales.find_all('a'):
        lista_notas.append(link.get('href'))
        
    # Obtengo las url de la lista de noticias
    lista_noticias = sopa.find('article', attrs={'class' : "article-item article-item--teaser"})
    for noticia in lista_noticias.find_all('a'):
        lista_notas.append(noticia.get('href'))
        
    return lista_notas

Al analizar la lista que nos devuelve la función advertimos que falta la dirección httml y tenemos muchos links repetidos, esto es devido a que no solamente tenemos la dirección url en el título sino también en el resumen y en algunos casos también en la imagen. Asimismo, tenemos algunos links a los perfiles de los y las periodistas que redactaron las notas.\
Nos va a tocar limpiar el contenido de la lista.\
Ahora que tenemos la nueva lista limpia, y sin los links https, vamos a agregar a cada elemento de la lista la dirección de la web de página12.\
Lo hacemos con un ciclo for que itere por cada elemento de la nueva lista, y guardamos el resultado en otra nueva lista. Luego creamos un función para automatizar el proceso y ya estamos listos para continuar con el scrap.

In [27]:
# Para limpiar la lista que nos devuelve la función vamos a usar pandas, para facilitarlo.
lista_notas = extraer_notas(sopa_seccion)
#len(lista_notas)
lista_notas = list(set(lista_notas))

#type(lista_notas)
lista_nueva_notas = []
lista_links_notas = []
for elemento in lista_notas:
    elemento = str(elemento)
    if elemento.find('http') and elemento.find('None'):
        lista_nueva_notas.append(elemento)
        for nuevo_elemento in lista_nueva_notas:
            elemento = str(elemento)
        lista_links_notas.append ('https://www.pagina12.com.ar' + elemento)
        
lista_links_notas

['https://www.pagina12.com.ar/499641-hebe-de-bonafini-la-despedida-de-los-organismos-de-derechos-',
 'https://www.pagina12.com.ar/499895-argentina-recupero-archivos-diplomaticos-relacionados-con-la',
 'https://www.pagina12.com.ar/499699-las-reacciones-de-l',
 'https://www.pagina12.com.ar/499879-los-aliados-del-pro-y-un-intento-de-borrar-la-memoria',
 'https://www.pagina12.com.ar/499760-los-pagos-de-caputo-hermanos-a-revolucion-federal-el-juez-pi',
 'https://www.pagina12.com.ar/499882-caba-de-dignidades-y-sumisiones',
 'https://www.pagina12.com.ar/499785-katopodis-revelo-que-les-dijo-cristina-kirchner-a-los-dirige',
 'https://www.pagina12.com.ar/secciones/el-pais?page=1',
 'https://www.pagina12.com.ar/499903-la-apuesta-del-kirchnerismo-un-salto-del-pasado-al-futuro',
 'https://www.pagina12.com.ar/499653-ella-tambien',
 'https://www.pagina12.com.ar/499808-zurro-lo-de-macri-fue-una-pseudo-democracia-con-practicas-ma',
 'https://www.pagina12.com.ar/499771-la-carta-del-papa-francisco-por-la

### Corrección y manejo de errores:
Ahora lo que debemos realizar es continuar con el scraping y generar una función que realice la misma acción previa en cada una de las secciones, con el problema que nos encontramos, fue que al realizarla, y al haber algunos links caídos, se nos cortaba la ejecución del ciclo for. Al encontrarme con este error, lo que advertí fue la manejar y procesar los errores.\
Para manejar los errores podemos utilizar la librería request, y crear una condicional, habíamos dicho que si estaba todo ok, nos devolvía el código 200, por tal manera podemos crear una conición if que nos devuelva el booleano de comparación.

In [28]:
r = requests.get(lista_links_notas[0])

if r.status_code == 200:
    #procesamos la respuesta
    print ('procesamos')
else:
    print ('informamos el error...')
    #informar el error
try:
    requests.get(url_caida)
except Exception as e:
    print ('Error al hacer requests')
    print (e)
    print ('\n')

procesamos
Error al hacer requests
name 'url_caida' is not defined




In [29]:
try:
    art_top.a.get('href')
except:
    pass

In [30]:
r = requests.get(links_secciones[0])
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos...')

procesamos..


### Extracción contenido de cada nota
Ahora que ya tenemos los links a cada una de las notas de cada sección guardados en una lista, vamos a proceder a acceder al contenido de cada nota y extraer todo lo que querramos.\
Para ello lo primero que vamos a hacer es quedarnos con la primer url y guardarlo en una variable, para trabajar más tranquilos.

In [31]:
url_nota = lista_links_notas[0]
url_nota

'https://www.pagina12.com.ar/499641-hebe-de-bonafini-la-despedida-de-los-organismos-de-derechos-'

In [32]:
try:
    nota = requests.get(url_nota)
    if nota.status_code == 200:
        sopa_nota = BeautifulSoup(nota.text, 'lxml')
        # Extraemos el título
        titulo = sopa_nota.find('div', attrs={'class' : 'col 2-col'}).find('h1')
        volanta = sopa_nota.find('div', attrs= {'class' : 'col 2-col'}).find('h4')
        copete = sopa_nota.find('div', attrs= {'class' : 'col 2-col'}).find('h3')
        fecha = sopa_nota.find('div', attrs= {'class' : 'date modification-date'}).find('time')
        autor = sopa_nota.find('div', attrs= {'class' : 'authors-inner-list'}).find('a')
        cuerpo = sopa_nota.find('div', attrs = {'class' : 'article-main-content article-text'})
        #imagen = 
        print('El título es: ', titulo.text)
        print ('La volanda es: ', volanta.text)
        print ('El copete dice: ', copete.text)
        print ('Fue escrita el día: ', fecha.text)
        print ('Por el autor: ', autor.text)
        print (cuerpo.text)
except Exception as e:
    print ('Error: ')
    print (e)
    print('\n')

El título es:  Hebe de Bonafini: la despedida de los organismos de derechos humanos
La volanda es:  El recuerdo de quienes compartieron la lucha contra la impunidad 
El copete dice:  
Fue escrita el día:  21 de noviembre de 2022 - 00:57
Por el autor:  Luis Corbacho
“Siempre puso el cuerpo”. “La recordamos en la plaza enfrentando a la dictadura y también en el 2001”. “Aprendimos a caminar con ella”.  “Sembró mucho”, fueron algunas de las palabras de despedida expresadas desde los organismos de derechos humanos, donde no faltaron recuerdos sobre el carácter fuerte de Hebe de Bonafini y las diferencias a lo largo de 46 años de militancia.  Su dimensión histórica también quedó impresa en eso: no sólo en las lucha que dió, sino en la densidad de sus posturas y en las generaciones que fue hilando, cada una con sus nuevas batallas. Su hija Alejandra había sido quien, desde la familia, comunicó la muerte de Hebe, ocurrida a las 9.20 de la mañana en el hospital Italiano de La Plata. Contó que e

Ahora que ya extrajimos la información que queremos de toda la nota, vamos a crear una función para que lo replique a las demás notas y a todas las demás secciones.\
Luego hacemos una función unificando todo y tendríamos listo el proyecto.

In [33]:
def extraer_contenido_nota (sopa):
    
    # Creamos un diccionario para guardar toda la información
    diccionario_notas = {}
    
    # Extraemos la fecha
    fecha = sopa.find('div', attrs= {'class' : 'date modification-date'})
    if fecha:
        diccionario_notas['fecha'] = fecha.text
    else:
        diccionario_notas['fecha'] = None
    
    # Extraemos la volanta
    volanta = sopa.find('div', attrs= {'class' : 'section-2-col article-header'})
    if volanta:
        for volanta in volanta.find_all('h4'):
            diccionario_notas['volanta'] = volanta.text
    else:
        diccionario_notas['volanta'] = None
    
    # Extraemos el título
    titulo = sopa.find('div', attrs={'class' : 'section-2-col article-header'})
    if titulo:
        for titulo in titulo.find_all ('h1'):
            diccionario_notas['titulo'] = titulo.text
    else:
        diccionario_notas['titulo'] = None
    
    # Extraemos el copete
    copete = sopa.find('div', attrs= {'class' : 'section-2-col article-header'})
    if copete:
        for copete in copete.find_all('h3'):
            diccionario_notas['copete'] = copete.text
    else:
        diccionario_notas['copete'] = None
    
    # Extraemos el cuerpo
    cuerpo = sopa.find('div', attrs = {'class' : 'article-main-content article-text'})
    if cuerpo:
        diccionario_notas['cuerpo'] = cuerpo.text
    else:
        diccionario_notas['cuerpo'] = None
    
    # Extraemos el autor
    autor = sopa.find('div', attrs= {'class' : 'authors-inner-list'})
    if autor:
        diccionario_notas['autor'] = autor.text
    else:
        diccionario_notas['autor'] = None
        
    return diccionario_notas

In [34]:
extraer_contenido_nota (sopa_nota)

{'fecha': '21 de noviembre de 2022 - 00:57',
 'volanta': 'El recuerdo de quienes compartieron la lucha contra la impunidad\xa0',
 'titulo': 'Hebe de Bonafini: la despedida de los organismos de derechos humanos',
 'copete': '',
 'cuerpo': '“Siempre puso el cuerpo”. “La recordamos en la plaza enfrentando a la dictadura y también en el 2001”. “Aprendimos a caminar con ella”.\xa0 “Sembró mucho”, fueron algunas de las palabras de despedida expresadas desde los organismos de derechos humanos, donde no faltaron recuerdos sobre el carácter fuerte de Hebe de Bonafini y las diferencias a lo largo de 46 años de militancia.\xa0 Su dimensión histórica también quedó impresa en eso: no sólo en las lucha que dió, sino en la densidad de sus posturas y en las generaciones que fue hilando, cada una con sus nuevas batallas.\xa0Su hija Alejandra había sido quien, desde la familia, comunicó la muerte de Hebe, ocurrida a las 9.20 de la mañana en el hospital Italiano de La Plata. Contó que en los últimos días

In [35]:
def scraping_nota (url):
    try:
        nota = requests.get (url)
    except Exception as e:
        print ('Error en el scraping por la URL', url)
        print (e)
        print ('\n')
        return None
    
    if nota.status_code != 200:
        print(f'Error obteniendo la nota {url}')
        print (f'status Code: {nota.status_code}')
        return None
    
    sopa_nota = BeautifulSoup (nota.text, 'lxml')
    
    diccionario_csv = extraer_contenido_nota (sopa_nota)
    diccionario_csv['url'] = url
    
    return diccionario_csv

In [36]:
scraping_nota (url_nota)

{'fecha': '21 de noviembre de 2022 - 00:57',
 'volanta': 'El recuerdo de quienes compartieron la lucha contra la impunidad\xa0',
 'titulo': 'Hebe de Bonafini: la despedida de los organismos de derechos humanos',
 'copete': '',
 'cuerpo': '“Siempre puso el cuerpo”. “La recordamos en la plaza enfrentando a la dictadura y también en el 2001”. “Aprendimos a caminar con ella”.\xa0 “Sembró mucho”, fueron algunas de las palabras de despedida expresadas desde los organismos de derechos humanos, donde no faltaron recuerdos sobre el carácter fuerte de Hebe de Bonafini y las diferencias a lo largo de 46 años de militancia.\xa0 Su dimensión histórica también quedó impresa en eso: no sólo en las lucha que dió, sino en la densidad de sus posturas y en las generaciones que fue hilando, cada una con sus nuevas batallas.\xa0Su hija Alejandra había sido quien, desde la familia, comunicó la muerte de Hebe, ocurrida a las 9.20 de la mañana en el hospital Italiano de La Plata. Contó que en los últimos días

In [37]:
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo']

In [38]:
notas = []
notas_limpias = []
notas_limpias_link = []

for link in links_secciones:
    try:
        r = requests.get (link)
        if r.status_code == 200:
            sopa = BeautifulSoup (r.text, 'lxml')
            notas.extend(extraer_notas(sopa))
            for elemento in notas:
                elemento = str(elemento)
                if elemento.find('http') and elemento.find('None'):
                    notas_limpias.append(elemento)
                    for nuevo_elemento in notas_limpias:
                        elemento = str(elemento)
                        notas_limpias_link.append ('https://www.pagina12.com.ar' + elemento)
                        notas_limpias_link = list(set(notas_limpias_link))
            
        else:
            print ('No se pudo obtener la sección', link)
   
    except Exception as e:
        print ('Error: ')
        print (e)
        print('\n')

notas_limpias_link

Error: 
'NoneType' object has no attribute 'find_all'




['https://www.pagina12.com.ar/499912-crimen-de-luciano-olivera-cinco-policias-bonaerenses-van-a-j',
 'https://www.pagina12.com.ar/499638-mundial-qatar-2022-que-dijeron-los-duenos-de-la-pelota-en-la',
 'https://www.pagina12.com.ar/499607-messi-se-entreno-con-normalidad-y-despejo-dudas',
 'https://www.pagina12.com.ar/499882-caba-de-dignidades-y-sumisiones',
 'https://www.pagina12.com.ar/499877-mundial-qatar-2022-a-estados-unidos-se-le-escapo-la-tortuga-',
 'https://www.pagina12.com.ar/499785-katopodis-revelo-que-les-dijo-cristina-kirchner-a-los-dirige',
 'https://www.pagina12.com.ar/499597-el-triunfo-de-lula-anima-a-la-region-a-relanzar-unasur',
 'https://www.pagina12.com.ar/499641-hebe-de-bonafini-la-despedida-de-los-organismos-de-derechos-',
 'https://www.pagina12.com.ar/499331-la-decada-pospandemia-viene-fulera-muy-fulera',
 'https://www.pagina12.com.ar/499567-donald-trump-el-candidato-inoxidable',
 'https://www.pagina12.com.ar/499874-entre-rios-continua-aumentando-el-caudal-del-rio-p

In [39]:
len(notas_limpias_link)

67

In [40]:
len (notas)


# Para limpiar la lista que nos devuelve la función vamos a usar pandas, para facilitarlo.
#lista_notas = extraer_notas(sopa_seccion)
#len(lista_notas)
#lista_notas = list(set(lista_notas))

#type(lista_notas)
lista_nueva_notas_todas = []
lista_links_notas_todas = []
for elemento in notas:
    elemento = str(elemento)
    if elemento.find('http') and elemento.find('None'):
        lista_nueva_notas_todas.append(elemento)
        for nuevo_elemento in lista_nueva_notas_todas:
            elemento = str(elemento)
        lista_links_notas_todas.append ('https://www.pagina12.com.ar' + elemento)

lista_links_notas_todas = list(set(lista_links_notas_todas))
lista_links_notas_todas

['https://www.pagina12.com.ar/499912-crimen-de-luciano-olivera-cinco-policias-bonaerenses-van-a-j',
 'https://www.pagina12.com.ar/499638-mundial-qatar-2022-que-dijeron-los-duenos-de-la-pelota-en-la',
 'https://www.pagina12.com.ar/499877-mundial-qatar-2022-a-estados-unidos-se-le-escapo-la-tortuga-',
 'https://www.pagina12.com.ar/499882-caba-de-dignidades-y-sumisiones',
 'https://www.pagina12.com.ar/499785-katopodis-revelo-que-les-dijo-cristina-kirchner-a-los-dirige',
 'https://www.pagina12.com.ar/499607-messi-se-entreno-con-normalidad-y-despejo-dudas',
 'https://www.pagina12.com.ar/499597-el-triunfo-de-lula-anima-a-la-region-a-relanzar-unasur',
 'https://www.pagina12.com.ar/499641-hebe-de-bonafini-la-despedida-de-los-organismos-de-derechos-',
 'https://www.pagina12.com.ar/499331-la-decada-pospandemia-viene-fulera-muy-fulera',
 'https://www.pagina12.com.ar/499567-donald-trump-el-candidato-inoxidable',
 'https://www.pagina12.com.ar/499874-entre-rios-continua-aumentando-el-caudal-del-rio-p

In [41]:
data = []

for i, nota in enumerate (notas_limpias_link):
    print(f'Estamos escrapendo la nota {i}/{len(notas_limpias_link)}')
    data.append(scraping_nota(nota))

Estamos escrapendo la nota 0/67
Estamos escrapendo la nota 1/67
Estamos escrapendo la nota 2/67
Estamos escrapendo la nota 3/67
Estamos escrapendo la nota 4/67
Estamos escrapendo la nota 5/67
Estamos escrapendo la nota 6/67
Estamos escrapendo la nota 7/67
Estamos escrapendo la nota 8/67
Estamos escrapendo la nota 9/67
Estamos escrapendo la nota 10/67
Estamos escrapendo la nota 11/67
Estamos escrapendo la nota 12/67
Estamos escrapendo la nota 13/67
Estamos escrapendo la nota 14/67
Estamos escrapendo la nota 15/67
Estamos escrapendo la nota 16/67
Estamos escrapendo la nota 17/67
Estamos escrapendo la nota 18/67
Estamos escrapendo la nota 19/67
Estamos escrapendo la nota 20/67
Estamos escrapendo la nota 21/67
Estamos escrapendo la nota 22/67
Estamos escrapendo la nota 23/67
Estamos escrapendo la nota 24/67
Estamos escrapendo la nota 25/67
Estamos escrapendo la nota 26/67
Estamos escrapendo la nota 27/67
Estamos escrapendo la nota 28/67
Estamos escrapendo la nota 29/67
Estamos escrapendo l

In [42]:
len(data)

67

In [43]:
df = pd.DataFrame(data)


In [44]:
df.head(25)

,fecha,volanta,titulo,copete,cuerpo,autor,url
0,21 de noviembre de 2022 - 19:37,"Uno de los agentes está acusado de homicidio ""...",Crimen de Luciano Olivera: cinco policías bona...,"El principal acusado, Maximiliano González, po...",Cinco integrantes de la Policía bonaerense irá...,Por Luis Corbacho,https://www.pagina12.com.ar/499912-crimen-de-l...
1,21 de noviembre de 2022 - 00:56,"Entre discursos,\nemires y mascotas",Mundial Qatar 2022: qué dijeron los dueños de ...,El líder qatarí y el presidente de la FIFA apr...,"Shows musicales, danzas coreografiadas, muñeco...",Por Luis Corbacho,https://www.pagina12.com.ar/499638-mundial-qat...
2,21 de noviembre de 2022 - 00:57,"El astro había descansado el sábado, pero está...",Messi se entrenó con normalidad y despejó dudas,Tranquilidad para Scaloni: La penúltima prácti...,"A tan poco tiempo del debut, cualquier situaci...",Por Luis Corbacho,https://www.pagina12.com.ar/499607-messi-se-en...
3,21 de noviembre de 2022 - 18:45,,CABA: De dignidades y sumisiones,,El triunfo de la lucha de los jóvenes médicos ...,Por Luis Corbacho,https://www.pagina12.com.ar/499882-caba-de-dig...
4,21 de noviembre de 2022 - 18:31,"Los norteamericanos ganaban, pero el empate de...",Mundial Qatar 2022: a Estados Unidos se le esc...,La superioridad del elenco yanqui fue abrumado...,El debut de Estados Unidos este lunes en el Mu...,Por Luis Corbacho,https://www.pagina12.com.ar/499877-mundial-qat...
5,21 de noviembre de 2022 - 13:08,"""Hay que poner el foco en las preocupaciones d...",Katopodis reveló qué les dijo Cristina Kirchne...,,"El ministro de Obras Públicas, Gabriel Katopod...",Por Luis Corbacho,https://www.pagina12.com.ar/499785-katopodis-r...
6,21 de noviembre de 2022 - 00:58,Siete de sus doce miembros se retiraron del or...,El triunfo de Lula anima a la región a relanza...,"""Con una Guerra Fría que se aproxima a pasos ...",El triunfo de Luiz Inácio Lula da Silva en Bra...,Por Luis Corbacho,https://www.pagina12.com.ar/499597-el-triunfo-...
7,21 de noviembre de 2022 - 00:57,El recuerdo de quienes compartieron la lucha c...,Hebe de Bonafini: la despedida de los organism...,,“Siempre puso el cuerpo”. “La recordamos en la...,Por Luis Corbacho,https://www.pagina12.com.ar/499641-hebe-de-bon...
8,21 de noviembre de 2022 - 11:55,EXCLUSIVO. Encuesta mundial 2022 a trabajadore...,"La década pospandemia viene fulera, muy fulera",La vicepresidenta CFK advirtió sobre el escena...,La vicepresidenta Cristina Fernández de Kirchn...,Por Luis Corbacho,https://www.pagina12.com.ar/499331-la-decada-p...
9,21 de noviembre de 2022 - 00:58,Opinión,"Donald Trump, el candidato inoxidable",,Las elecciones de medio término que tuvieron l...,Por Luis Corbacho,https://www.pagina12.com.ar/499567-donald-trum...


In [45]:
df.to_csv('scraping_notas_pagina12-21-11-22.csv')